In [2]:
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from workflow.scripts.plotting_tools import global_map
import numpy as np
import matplotlib as mpl
import xesmf as xe

In [9]:
paths = sorted(snakemake.input.paths)
params = snakemake.params
time_slice = params.get('time_slice', slice(3,-1))

In [7]:
ds = xr.open_dataset(snakemake.input.paths[0])

In [8]:
ds

In [10]:
dsets = {}
for path in paths:
    ds = xr.open_dataset(path)
    source_id = ds.source_id
    ds = ds[snakemake.wildcards.vName].isel(year=time_slice).mean(dim='year',keep_attrs=True).to_dataset(name=f'{source_id}')
    dsets[source_id]=ds
    

In [11]:
fig,ax = plt.subplot_mosaic([['a)','b)','c)'],['d)','e)','f)'],['g)','h)','i)']], 
                            subplot_kw={'projection':ccrs.PlateCarree()}, figsize=(18,10), sharex=True, sharey=True)
norm = mpl.colors.TwoSlopeNorm(vcenter=0, vmax=11, vmin=-5)
for k, i in zip(ax.keys(), dsets.keys()):
    pfs = dsets[i][i].plot.contourf(ax=ax[k],levels=24,norm=norm,cmap='RdYlBu_r', add_colorbar=False)
    ax[k].coastlines()
    ax[k].set_title(i)
    global_map(ax[k])
cax = fig.add_axes([0.94,0.2,0.02,0.62])
cmap = mpl.cm.RdYlBu_r
fig.colorbar(pfs, cax=cax, extend='both', 
             label='$W/m^2$')
plt.savefig(snakemake.output.outpath, bbox_inches='tight', facecolor='white')